In [1]:
import pandas as pd
from decimal import *
import numpy as np
from geopy.distance import vincenty

In [2]:
xl = pd.ExcelFile("./data/donneesBrutesPirateriesIssuesInternet.xlsx")
shorlineAden = pd.read_csv('./data/shorlineAden.csv',sep=';')

In [3]:
xl.sheet_names

[u'Feuil1']

In [4]:
df = xl.parse("Feuil1")

In [5]:
shorlineAden.head()

,Unnamed: 0,longitude,latitude,id
0,358,"53,9743183","26,497247",25208
1,368,"53,9697719","26,4984432",25208
2,378,"53,9669274","26,4991375",25208
3,388,"53,9644766","26,4990889",25208
4,398,"53,9611029","26,5011279",25208


In [6]:
df=df.drop(df.index[0])

In [7]:
def dmsToDd(dms):  
    #remove the last character : N S E W
    direction=dms[-1:]
    dms = dms[:-1]
    
    tab=dms.split(':', 1 )
    degree=tab[0]
    minuteSeconde=tab[1]
    #print degree
    #print minuteSeconde
    if ":" in minuteSeconde:
        minuteSeconde=minuteSeconde.replace(":", ".")
    #print "-------------"
    #print degree
    #print minuteSeconde
    lastChar=minuteSeconde[-1:]
    if lastChar==".":
        minuteSeconde=minuteSeconde[:-1]
    
    if "." in minuteSeconde:
        tab2=minuteSeconde.split('.', 1 )
        minute=tab2[0]
        seconde=tab2[1]
        #print minuteSeconde
        #print minute
        #print seconde
        result=float(degree)+(float(minute)/60)+(float(seconde)/3600) 
    else:
        result=float(degree)+(float(minuteSeconde)/60)
    
    if direction=="S" or direction=="W":
        result=-result
    return result
    

dms="09:24.6S"

toto =dmsToDd(dms)
print toto
print format(toto, '.6f')
print round(toto, 4)

-9.40166666667
-9.401667
-9.4017


In [8]:
df['latitude'] = df.apply(lambda row: dmsToDd(row['lat (0)']), axis=1)
df['longitude'] = df.apply(lambda row: dmsToDd(row['long(0)']), axis=1)

In [36]:
dfAden=df[(df.latitude<33.) & (df.latitude>-13.) & (df.longitude>30.) & (df.longitude<77.)]
print len(dfAden)
print len(df)

924
2188


In [37]:
shorlineAden=shorlineAden.replace({',': '.'}, regex=True)

In [40]:
lon=50.42038
lat=4.039618
threshold=100
def distance(lat, lon):
    tab=[]
    pointAttack=(lat, lon)
    shorlineAdenCurrent=shorlineAden[(shorlineAden.latitude<lat+threshold) & (shorlineAden.latitude>lat-threshold) & (shorlineAden.longitude>lon-threshold) & (shorlineAden.longitude<lon+threshold)]
    print len(shorlineAdenCurrent)
    for index in range(0,2):
        #if index%10000==0:
        #    print index
        #print shorlineAden.iloc[index]['x0']
        pointCoast= (shorlineAden.iloc[index]['latitude'], shorlineAden.iloc[index]['longitude'])
        distance =vincenty(pointCoast, pointAttack).kilometers
        #print distance
        tab.append(distance)
     
    return min(tab)

print 'distance : ' +str(distance(lat, lon))

0
distance : 2513.93353557


In [ ]:
newport_ri = (41.49008, -71.312796)
cleveland_oh = (41.499498, -81.695391)
print(vincenty(newport_ri, cleveland_oh).miles)

In [ ]:
dfAden['distance_shorline']=dfAden.apply(lambda row: distance(row['latitude'], row['longitude']), axis=1)

In [15]:
df.to_excel("./data/donneesBrutesPirateriesLatLon.xlsx", encoding='utf-8')